In [1]:
import sys,os
import random
import numpy as np
import json
from collections import OrderedDict
from gnn import GNNNet
from utils import *
from emetrics import *
from data_process import create_dataset_for_5folds
import torch
import torch.nn as nn
from torch_geometric.data import DataLoader
import rdkit as rd
from torch_sparse import SparseTensor,transpose
import deepchem
import tensorflow as tf
import pandas as pd
import pickle

### Training

In [2]:
datasets = [['davis', 'kiba'][1]]
datasets 

['kiba']

In [3]:
method=['pconsc4', 'esm_cmaps', 'alpha_fold_cmaps'][0]
method

'pconsc4'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.device('cuda:0')
cuda_name = ['cuda:0', 'cuda:1', 'cuda:2', 'cuda:3'][0]
print('cuda_name:', cuda_name)

cuda_name: cuda:0


In [5]:
fold = [0, 1, 2, 3, 4][0]
cross_validation_flag = True
# print(int(sys.argv[3]))

TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE = 128
LR = 0.001
NUM_EPOCHS = 1500

print('Learning rate: ', LR)
print('Epochs: ', NUM_EPOCHS)

Learning rate:  0.001
Epochs:  1500


In [6]:

models_dir = 'models_sample'
results_dir = 'results'

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(results_dir):
    os.makedirs(results_dir)

# Main program: iterate over different datasets
result_str = ''
USE_CUDA = torch.cuda.is_available()
device = torch.device(cuda_name)
model_file_name = 'models_sample/model_kiba_pconsc4_500_GNNNet_kiba_0.model'
model = GNNNet()
model.to(device)
model.load_state_dict(torch.load(model_file_name, map_location=cuda_name))
# model = GNNNet()
# model=model_kiba_esm__188_500_GNNNet_kiba_1.model
#model.to(device)
model_st = GNNNet.__name__
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

GNNNet Loaded


In [ ]:
for dataset in datasets:
    train_data, valid_data = create_dataset_for_5folds(dataset, fold, method)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=True,
                                               collate_fn=collate)
    valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=TEST_BATCH_SIZE, shuffle=False,
                                               collate_fn=collate)

    best_mse = 1000
    best_test_mse = 1000
    best_epoch = -1
    model_file_name = 'models_sample/model_kiba_pconsc4_full_2000_' + model_st + '_' + dataset + '_' + str(fold) + '.model'
    mse_list1=[]
    for epoch in range(NUM_EPOCHS):
        train(model, device, train_loader, optimizer, epoch + 1)
        print('predicting for valid data')
        G, P = predicting(model, device, valid_loader)
        val = get_mse(G, P)
        mse_list1.append(val)
        print('valid result:', val, best_mse)
        if val < best_mse:
            best_mse = val
            best_epoch = epoch + 1
            torch.save(model.state_dict(), model_file_name)
            print('rmse improved at epoch ', best_epoch, '; best_test_mse', best_mse, model_st, dataset, fold)
        else:
            print('No improvement since epoch ', best_epoch, '; best_test_mse', best_mse, model_st, dataset, fold)


dataset: kiba
fold: 0
train entries: 78836 effective train entries 78717
valid entries: 19709 effective valid entries 19685
effective drugs,effective prot: 2068 228


/disk/scratch/DGraphDTA/utils.py:52: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  GCNData_mol = DATA.Data(x=torch.Tensor(features),


In [ ]:
import matplotlib.pyplot as plt
ep=[i for i in range(1,1501)]

plt.plot(ep,mse_list1)

In [ ]:
with open('mse_pconcs4_full_2000_file.txt', 'w') as f:
    for item in mse_list1:
        f.write("%s\n" % item)

In [ ]:
train_fold_origin = json.load(open('data/kiba/' + 'folds/train_fold_setting1.txt'))
train_fold_origin = [e for e in train_fold_origin]

In [ ]:
train_fold_origin

In [ ]:
len(train_fold_origin)

In [ ]:
test_fold_origin = json.load(open('data/kiba/' + 'folds/test_fold_setting1.txt'))
test_fold_origin = [e for e in test_fold_origin]


In [ ]:
np.sort(test_fold_origin)

In [ ]:
flat_list = [item for sublist in train_fold_origin for item in sublist]
flat_list 

In [ ]:
len(flat_list)

In [ ]:
np.sort(flat_list)

In [ ]:
np.sort(flat_list)

In [ ]:
len(flat_list)

In [ ]:
len(train_fold_origin[2])

In [ ]:
train_fold_origin[2]

In [ ]:
with open('data/kiba/kiba_1024.txt') as f:
    lines = f.readlines()
lines

In [ ]:
with open('data/kiba/kiba_1024.txt', "r") as fd:
    lines = fd.read().splitlines()
lines

In [ ]:
affinity = pickle.load(open( 'data/kiba/'+ 'Y', 'rb'), encoding='latin1')
affinity = np.asarray(affinity)
affinity

In [ ]:
affinity.shape

In [ ]:
affinity2 = np.load(open('data/kiba/' + 'Y_updated.npy', 'rb'))

In [ ]:
affinity2.shape

In [ ]:
affinity2

In [ ]:
len(np.where(np.isnan(affinity2.flatten()) == False)[0])

In [ ]:
2111*229

In [ ]:
flat_aff=affinity.flatten()
len(flat_aff)

In [ ]:
len(np.where(np.isnan(flat_aff) == False)[0])

In [ ]:
import pandas as pd
df4=pd.read_csv('data/kiba/kiba.csv')  
df4.head()

In [ ]:
df4.shape

In [ ]:
df4.columns

In [ ]:
np.unique(df4.index.values)

In [ ]:
len(np.unique(df4.index.values))

In [ ]:
df4['Unnamed: 0']

In [ ]:
df4.index.values

In [ ]:
df4

In [ ]:
#df4 = df4.set_index('Unnamed: 0')
df4

In [ ]:
import ast
file = open("data/kiba/ligands_can.txt", "r")

contents = file.read()
ligands = ast.literal_eval(contents)

file.close()

print(type(ligands))


print(ligands)

In [ ]:
ligands.keys()

In [ ]:
kiba_p.keys()

In [ ]:
proteins_upd=kiba_p

In [ ]:
len(proteins_upd)

In [ ]:
import json
  
with open("data/kiba/proteins_updated.txt", 'w') as convert_file:
     convert_file.write(json.dumps(proteins_upd))

In [ ]:
for i in lines:
    del proteins_upd[i]

In [ ]:
len(ligands.keys())

In [ ]:
import ast
file = open("data/kiba/proteins.txt", "r")

contents = file.read()
kiba_p = ast.literal_eval(contents)

file.close()

print(type(kiba_p))


print(kiba_p)


In [ ]:
kiba_p.keys()

In [ ]:
header

In [ ]:
import csv
file = open('data/kiba/kiba.csv')
csvreader = csv.reader(file)

In [ ]:
t2=set(kiba_p.keys())

In [ ]:
ligand.ke

In [ ]:
#list(set(header))-list(set(kiba_p.keys()))
s = set(np.unique(df4.columns))
temp3 = [x for x in s if x not in t2]
np.sort(temp3)

In [ ]:
s = set(np.unique(df4.index.values))
temp4 = [x for x in s if x not in list(ligands.keys())]
np.sort(temp4)

In [ ]:
len(temp4)

In [ ]:
df6=df4.copy(deep=True)

In [ ]:
df6

In [ ]:
df6_a=df6.drop(labels=lines,axis=1)

In [ ]:
df6_a.shape

In [ ]:
df6=df6.drop(labels=temp3,axis=1)

In [ ]:
df6=df6.drop(labels=temp4,axis=0)

In [ ]:
df6

In [ ]:
len(np.where(np.isnan(df6_a.to_numpy().flatten()) == False)[0])

In [ ]:
data_full=np.where(np.isnan(df6_a.to_numpy().flatten()) == False)[0]

In [ ]:
np.where(np.isnan(df6_a.to_numpy().flatten()) == False)[0]

In [ ]:
np.sort(data_full)

In [ ]:
95574/6

In [ ]:
samples=[i for i in range(0,95574)]
len(samples)


In [ ]:
import random 
def partition (list_in, n):
    random.shuffle(list_in)
    return [list_in[i::n] for i in range(n)]

In [ ]:
parts_6=partition(samples,6)

In [ ]:
with open('data/kiba/folds/train_fold_setting2.txt','w') as myfile:
    json.dump(parts_6,myfile)

In [ ]:
parts_test=parts_6[5]

In [ ]:
with open('data/kiba/folds/test_fold_setting2.txt', 'w') as temp_file:
    json.dump(parts_test,temp_file)

In [ ]:
len(parts_6)

In [ ]:
len(parts_6[1])

In [ ]:
train_fold_origin = json.load(open('data/kiba/' + 'folds/train_fold_setting1.txt'))
train_fold_origin = [e for e in train_fold_origin]

In [ ]:
d6

In [ ]:
df6.to_csv('/disk/scratch/DGraphDTA/data/kiba/main.csv')  

In [ ]:
pickle.dump(d6, open("/disk/scratch/DGraphDTA/data/kiba/main.pkl", "w"))

In [ ]:
np.save("/disk/scratch/DGraphDTA/data/kiba/Y_updated",df6_a.to_numpy(),allow_pickle=True)

In [ ]:
len(temp3)

In [ ]:
174+229

In [ ]:
header = []
header = next(csvreader)
header

In [ ]:
118254-19709*5

In [ ]:
np.where(np.isnan(affinity) == False)

In [ ]:
rows, cols = np.where(np.isnan(affinity) == False)
            

In [ ]:
rows, cols = rows[train_folds], cols[train_folds]
            train_fold_entries = []
            for pair_ind in range(len(rows)):
                if not valid_target(prot_keys[cols[pair_ind]], dataset):  # ensure the contact and aln files exists
                    continue
                ls = []
                ls += [drugs[rows[pair_ind]]]
                ls += [prots[cols[pair_ind]]]
                ls += [prot_keys[cols[pair_ind]]]
                ls += [affinity[rows[pair_ind], cols[pair_ind]]]
                train_fold_entries.append(ls)
                valid_train_count += 1

In [ ]:
datasets = [['davis', 'kiba'][1]]
datasets 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.device('cuda:0')
cuda_name = ['cuda:0', 'cuda:1', 'cuda:2', 'cuda:3'][0]
print('cuda_name:', cuda_name)

In [ ]:
fold = [0, 1, 2, 3, 4][0]
cross_validation_flag = True
# print(int(sys.argv[3]))

TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE = 128
LR = 0.001
NUM_EPOCHS = 500

print('Learning rate: ', LR)
print('Epochs: ', NUM_EPOCHS)

In [ ]:
models_dir = 'models_sample'
results_dir = 'results'

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(results_dir):
    os.makedirs(results_dir)

# Main program: iterate over different datasets
result_str = ''
USE_CUDA = torch.cuda.is_available()
device = torch.device(cuda_name)
model = GNNNet()
model.to(device)
model_st = GNNNet.__name__
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
for dataset in datasets:
    train_data, valid_data = create_dataset_for_5folds(dataset, fold)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=TRAIN_BATCH_SIZE, shuffle=True,
                                               collate_fn=collate)
    valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=TEST_BATCH_SIZE, shuffle=False,
                                               collate_fn=collate)

    best_mse = 1000
    best_test_mse = 1000
    best_epoch = -1
    model_file_name = 'models_sample/model_kiba_pconsc4_500_' + model_st + '_' + dataset + '_' + str(fold) + '.model'
    mse_list1=[]
    for epoch in range(NUM_EPOCHS):
        train(model, device, train_loader, optimizer, epoch + 1)
        print('predicting for valid data')
        G, P = predicting(model, device, valid_loader)
        val = get_mse(G, P)
        mse_list1.append(val)
        print('valid result:', val, best_mse)
        if val < best_mse:
            best_mse = val
            best_epoch = epoch + 1
            torch.save(model.state_dict(), model_file_name)
            print('rmse improved at epoch ', best_epoch, '; best_test_mse', best_mse, model_st, dataset, fold)
        else:
            print('No improvement since epoch ', best_epoch, '; best_test_mse', best_mse, model_st, dataset, fold)


In [ ]:
print(model)

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
f

In [ ]:
torch.cuda.empty_cache()

In [ ]:
print(t,r,a,f)

In [ ]:
print(t,r,a,f)

In [ ]:
torch.cuda.mem_get_info()

In [ ]:
torch.cuda.memory_stats()

In [ ]:
torch.cuda.memory_summary()

In [ ]:
import matplotlib.pyplot as plt
ep=[i for i in range(1,501)]

plt.plot(ep,mse_list1)

In [ ]:
with open('mse_pconsc4_500_file.txt', 'w') as f:
    for item in mse_list1:
        f.write("%s\n" % item)

In [ ]:
import os
import sys
import torch
import numpy as np
from random import shuffle
import matplotlib.pyplot as plt
from torch_geometric.data import Batch

from emetrics import get_aupr, get_cindex, get_rm2, get_ci, get_mse, get_rmse, get_pearson, get_spearman
from utils import *
from scipy import stats
from gnn import GNNNet
from data_process import create_dataset_for_test


def predicting(model, device, loader):
    model.eval()
    total_preds = torch.Tensor()
    total_labels = torch.Tensor()
    print('Make prediction for {} samples...'.format(len(loader.dataset)))
    with torch.no_grad():
        for data in loader:
            data_mol = data[0].to(device)
            data_pro = data[1].to(device)
            # data = data.to(device)
            output = model(data_mol, data_pro)
            total_preds = torch.cat((total_preds, output.cpu()), 0)
            total_labels = torch.cat((total_labels, data_mol.y.view(-1, 1).cpu()), 0)
    return total_labels.numpy().flatten(), total_preds.numpy().flatten()


def load_model(model_path):
    model = torch.load(model_path)
    return model


def calculate_metrics(Y, P, dataset='davis'):
    # aupr = get_aupr(Y, P)
    cindex = get_cindex(Y, P)  # DeepDTA
    cindex2 = get_ci(Y, P)  # GraphDTA
    rm2 = get_rm2(Y, P)  # DeepDTA
    mse = get_mse(Y, P)
    pearson = get_pearson(Y, P)
    spearman = get_spearman(Y, P)
    rmse = get_rmse(Y, P)

    print('metrics for ', dataset)
    # print('aupr:', aupr)
    print('cindex:', cindex)
    print('cindex2', cindex2)
    print('rm2:', rm2)
    print('mse:', mse)
    print('pearson', pearson)

    result_file_name = 'results/result_pconsc4_500_' + model_st + '_' + dataset + '.txt'
    result_str = ''
    result_str += dataset + '\r\n'
    result_str += 'rmse:' + str(rmse) + ' ' + ' mse:' + str(mse) + ' ' + ' pearson:' + str(
        pearson) + ' ' + 'spearman:' + str(spearman) + ' ' + 'ci:' + str(cindex) + ' ' + 'rm2:' + str(rm2)
    print(result_str)
    open(result_file_name, 'w').writelines(result_str)


def plot_density(Y, P, fold=0, dataset='davis'):
    plt.figure(figsize=(10, 5))
    plt.grid(linestyle='--')
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.scatter(P, Y, color='blue', s=40)
    plt.title('density of ' + dataset, fontsize=30, fontweight='bold')
    plt.xlabel('predicted', fontsize=30, fontweight='bold')
    plt.ylabel('measured', fontsize=30, fontweight='bold')
    # plt.xlim(0, 21)
    # plt.ylim(0, 21)
    if dataset == 'davis':
        plt.plot([5, 11], [5, 11], color='black')
    else:
        plt.plot([6, 16], [6, 16], color='black')
    # plt.legend()
    plt.legend(loc=0, numpoints=1)
    leg = plt.gca().get_legend()
    ltext = leg.get_texts()
    plt.setp(ltext, fontsize=12, fontweight='bold')
    plt.savefig(os.path.join('results', dataset + '_pconsc4_500_' + str(fold) + '.png'), dpi=500, bbox_inches='tight')




In [ ]:

dataset = ['davis', 'kiba'][1]  # dataset selection
model_st = GNNNet.__name__
print('dataset:', dataset)
cuda_name = ['cuda:0', 'cuda:1', 'cuda:2', 'cuda:3'][0] # gpu selection
print('cuda_name:', cuda_name)

 

In [ ]:
TEST_BATCH_SIZE = 128
models_dir = 'models'
results_dir = 'results'

device = torch.device(cuda_name if torch.cuda.is_available() else 'cpu')
#model_file_name = 'models_sample/model1111_' + model_st + '_' + dataset + '.model'
model_file_name = 'models_sample/model_kiba_pconsc4_500_GNNNet_kiba_0.model'
result_file_name = 'results/resul_pconsc4_500_' + model_st + '_' + dataset + '.txt'

model = GNNNet()
model.to(device)
model.load_state_dict(torch.load(model_file_name, map_location=cuda_name))
test_data = create_dataset_for_test(dataset)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=TEST_BATCH_SIZE, shuffle=False,
                                              collate_fn=collate)

Y, P = predicting(model, device, test_loader)
calculate_metrics(Y, P, dataset)
    


In [ ]:
test_loader

In [ ]:
Y

In [ ]:
P

In [ ]:

my_file = open("alpha500_pred_file.txt", "r")
content = my_file.read()
content=content.splitlines()
content=[float(i) for i in content]
print(content)

In [ ]:
P1=np.asarray(content)

In [ ]:
P1

In [ ]:
plt.scatter(P,color='blue')

In [ ]:
type(content)

In [ ]:
P

In [ ]:
plot_density(P1, P, fold, dataset)

In [ ]:
with open('pconsc4_500_gt_file.txt', 'w') as f:
    for item in list(Y):
        f.write("%s\n" % item)

In [ ]:
with open('pconsc4_500_pred_file.txt', 'w') as f:
    for item in list(P):
        f.write("%s\n" % item)

In [ ]:
plot_density(Y, P, fold, dataset)

In [ ]:
def plot_density1(Y, P,P1, fold=0, dataset='davis'):
    plt.figure(figsize=(10, 5))
    plt.grid(linestyle='--')
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.scatter(P, Y, color='blue', s=40)
    plt.scatter(P1, Y, color='green', s=40)
    plt.title('density of ' + dataset, fontsize=30, fontweight='bold')
    plt.xlabel('predicted', fontsize=30, fontweight='bold')
    plt.ylabel('measured', fontsize=30, fontweight='bold')
    # plt.xlim(0, 21)
    # plt.ylim(0, 21)
    if dataset == 'davis':
        plt.plot([5, 11], [5, 11], color='black')
    else:
        plt.plot([6, 16], [6, 16], color='black')
    # plt.legend()
    plt.legend(loc=0, numpoints=1)
    leg = plt.gca().get_legend()
    ltext = leg.get_texts()
    plt.setp(ltext, fontsize=12, fontweight='bold')
    #plt.savefig(os.path.join('results', dataset + 'alpha v/s pconsc4_500_' + str(fold) + '.png'), dpi=500, bbox_inches='tight')


In [ ]:
plot_density1(Y, P,P1, fold, dataset)

In [ ]:
plot_density(Y, P, fold, dataset)

In [ ]:
plot_density(Y, P, fold, dataset)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='2, 3'